In [ ]:

#%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, callbacks

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Helper libraries
import numpy as np
import pandas as pd
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.utils import compute_class_weight
import math

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from pathlib import Path
import os.path
#import datetime, os

from keras.applications import VGG16, VGG19, ResNet50, InceptionV3, InceptionResNetV2, Xception, MobileNet, DenseNet121, NASNetMobile, EfficientNetB0, MobileNetV2, MobileNetV3Large
from tensorflow.python.keras.layers import Conv2D, Flatten, Dropout, Dense, MaxPooling2D
from keras.models import load_model
from keras.utils.vis_utils import plot_model


In [ ]:
batch_size = 16
img_size = [240, 320]

# df = pd.read_csv(r'machine-learning-in-science-ii-2023/training_norm.csv')
# df['filename'] = df["image_id"].astype(str) + ".png"
# df = df.sample(frac=1).reset_index(drop=True)
# df.to_csv("training_norm_shuffle.csv", encoding='utf-8', index=False)

df = pd.read_csv(r'machine-learning-in-science-ii-2023/training_norm_shuffle.csv')
print(df)

x_train = df[0:int(len(df) * 0.6)]
x_validate = df[int(len(df) * 0.6):int(len(df) * 0.8)]
x_evaluate = df[int(len(df) * 0.8):]

# x_train = df[0:int(len(df) * 0.8)]
# x_validate = df[int(len(df) * 0.8):]


In [ ]:

shift = 0.1

training_datagen = ImageDataGenerator(
    rescale = 1./255,
    fill_mode='nearest',
    # width_shift_range=shift,
    # height_shift_range=shift
)

speed_train_generator = training_datagen.flow_from_dataframe(
    dataframe=x_train,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["speed"],
    target_size=img_size,
    batch_size=batch_size,
    shuffle=False,
    class_mode='other')

angle_train_generator = training_datagen.flow_from_dataframe(
    dataframe=x_train,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["angle"],
    target_size=img_size,
    batch_size=batch_size,
    shuffle=False,
    class_mode='other')

validation_datagen = ImageDataGenerator(rescale = 1./255)

speed_val_generator = validation_datagen.flow_from_dataframe(
    dataframe=x_validate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["speed"],
    target_size=img_size,
    shuffle=False,
    class_mode='other')

angle_val_generator = validation_datagen.flow_from_dataframe(
    dataframe=x_validate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["angle"],
    target_size=img_size,
    shuffle=False,
    class_mode='other')

evaluate_datagen = ImageDataGenerator(rescale = 1./255)

speed_eval_generator = evaluate_datagen.flow_from_dataframe(
    dataframe=x_evaluate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["speed"],
    target_size=img_size,
    shuffle=False,
    class_mode='other')

angle_eval_generator = evaluate_datagen.flow_from_dataframe(
    dataframe=x_evaluate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["angle"],
    target_size=img_size,
    shuffle=False,
    class_mode='other')



In [ ]:
filename = os.listdir("machine-learning-in-science-ii-2023/test_data/test_data") 

df = pd.DataFrame(filename)
df.columns = ["filename"]

df[['file', 'type']] = df.filename.str.split(".", expand = True)
df["file"] = df["file"].astype(str).astype(int)

df.sort_values(by=['file'], inplace=True)
df.reset_index(drop=True, inplace=True)
print(df)
test_datagen = ImageDataGenerator(rescale = 1./255)

test_images = test_datagen.flow_from_dataframe(
    dataframe=df,
    directory="machine-learning-in-science-ii-2023/test_data/test_data",
    x_col="filename",
    target_size=img_size,
    shuffle=False,
    class_mode=None)

In [ ]:
shape = (*img_size, 3) # inherited image size with 3 color filters
input_shape = [240, 320, 3]
transfer = MobileNetV2(
    input_shape=input_shape, 
    include_top=False, 
    weights='imagenet',
    classifier_activation="relu"
)

for layer in transfer.layers[:-2]:
    layer.trainable = False

In [ ]:
# Speed CNN base
speed_model = models.Sequential()

speed_model.add(transfer)
speed_model.build()
        
speed_model.add(layers.Conv2D(128, (3, 3), strides=(2,2), activation='relu'))
speed_model.add(layers.BatchNormalization())

speed_model.add(layers.Flatten()),

speed_model.add(layers.Dense(128, activation='relu'))
speed_model.add(layers.BatchNormalization())
speed_model.add(layers.Dropout(0.2))

speed_model.add(layers.Dense(64, activation='relu'))
speed_model.add(layers.BatchNormalization())
speed_model.add(layers.Dropout(0.2))

# Output layer
speed_model.add(layers.Flatten())
speed_model.add(layers.Dense(1,  activation='sigmoid', kernel_initializer='normal'))

In [ ]:
# Angle CNN base
angle_model = models.Sequential()

angle_model.add(transfer)
angle_model.build()
        
angle_model.add(layers.Conv2D(128, (3, 3), strides=(2,2), activation='relu'))
angle_model.add(layers.BatchNormalization())

angle_model.add(layers.Flatten()),

# model_1.add(layers.Dense(128, activation='relu'))
angle_model.add(layers.Dense(128, activation='relu'))
angle_model.add(layers.BatchNormalization())
angle_model.add(layers.Dropout(0.2))

# Output layer
angle_model.add(layers.Flatten())
angle_model.add(layers.Dense(1,  activation='relu', kernel_initializer='normal'))

In [ ]:
speed_model.build()
angle_model.build()

In [ ]:
#speed_model.summary()
#angle_model.summary()

In [ ]:
speed_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.001),
    loss='mean_squared_error'
)

angle_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.001),
    loss='mean_squared_error'
)

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=20,
    verbose=1,
    mode='min',
    restore_best_weights=True
)

In [ ]:
history = speed_model.fit(
    speed_train_generator,
    batch_size=batch_size,
    validation_data=speed_val_generator,
    callbacks=[es],
    epochs=100
)

In [ ]:
test_loss = speed_model.evaluate(
    speed_eval_generator,
    verbose=1
)

In [ ]:
history = angle_model.fit(
    angle_train_generator,
    batch_size=batch_size,
    validation_data=angle_val_generator,
    callbacks=[es],
    epochs=100
)

In [ ]:
test_loss = angle_model.evaluate(
    angle_eval_generator,
    verbose=1
)

In [ ]:
angle_prediction = angle_model.predict(test_images)
speed_prediction = speed_model.predict(test_images)

speed_prediction += 0.5
speed_prediction = np.floor(speed_prediction)

submission = {"angle":angle_prediction , "speed":speed_prediction}

df = pd.DataFrame(submission)
df.index += 1 
df.to_csv('Submission.csv')


In [ ]:
diff = abs(x_evaluate["angle"] - a)
results = pd.DataFrame({"Train":x_evaluate["angle"], "Predicted":a, "Difference":diff})
#results

In [ ]:
results = pd.DataFrame({"Train":x_evaluate["speed"],"Predicted":b})
#results

In [ ]:
speed_model.save("speed-model.h5")
angle_model.save("angle-model.h5")

In [ ]:
model2 = load_model('model.hdf5')

transfer = model2.layers[0]
transfer.trainable = True


for layer in transfer.layers[:-25]:
    layer.trainable = False

#model2.trainable = True

# for layer in model2.layers[:2]:
#     layer.trainable = False
    
model2.summary()


In [ ]:

model2.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.00001), # 1e-5 
    loss='mean_squared_error'
)

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.0005,
    patience=30,
    verbose=1,
    mode='min',
    restore_best_weights=True
)

history = model2.fit(
    train_generator,
    batch_size=batch_size,
    validation_data=val_generator,
    callbacks=[es],
    epochs=300
)


In [ ]:
from sklearn.metrics import classification_report

test_loss = model2.evaluate(
    eval_generator,
    verbose=1
)


In [ ]:
model2.save("model-fine-tuning.hdf5")

In [ ]:

prediction = model2.predict(test_images)
print(prediction.shape)
import math

a= prediction[:,0]
b= prediction[:,1]
b+=0.5
b= np.floor(b)

submissiondata={"angle":a , "speed":b} 
df = pd.DataFrame(submissiondata)
df.index += 1 
df.to_csv('Submission-fine-tuning.csv')


In [ ]:
model = load_model("model-fine-tuning.hdf5")
plot_model(model, show_layer_names=True, show_shapes=True,show_layer_activations=True)